In [32]:
homepath = ".."

# import python packages
import geopandas as gpd
from shapely.geometry import LineString
import os
import yaml
import json
from src import graphedit

In [33]:
# load configs
configfile = os.path.join(homepath, "config.yml")  # filepath of config file
configs = yaml.load(open(configfile), Loader=yaml.FullLoader)
proj_crs = configs["proj_crs"]  # projected CRS
municipality_codes = configs["municipalities"]

# input data
nodepath = homepath + "/data/raw/network/nodes.gpkg"
edgepath = homepath + "/data/raw/network/edges.gpkg"

### STUDY AREA

# define location of study area polygon (user-provided)
filepath_study = homepath + "/data/user_input/study_area.gpkg"

# read in study area polygon provided by the user
study_area = gpd.read_file(filepath_study)
study_area = study_area.to_crs(proj_crs)

### LOAD INPUT DATA AND PROJECT

nodes = gpd.read_file(nodepath)
edges = gpd.read_file(edgepath)

print("data read in!")

### LIMIT INPUT DATA TO STUDY AREA EXTENT

nodes.to_crs(proj_crs, inplace=True)
edges.to_crs(proj_crs, inplace=True)

assert nodes.crs == study_area.crs
assert edges.crs == study_area.crs

# only keep those nodes and edges that are within the study area
nodes = nodes[nodes.intersects(study_area.loc[0, "geometry"])].copy()
nodes.reset_index(drop=True, inplace=True)

edges = edges[edges.intersects(study_area.loc[0, "geometry"])].copy()
edges.reset_index(drop=True, inplace=True)

### PROCESS INPUT DATA TO FIND AND REMOVE PARALLEL EDGES

# create distinct column names for unique id
edges["edge_id"] = edges.id
nodes["node_id"] = nodes.id

data read in!


In [34]:
# assign edges initial start and end nodes
edges = graphedit.assign_edges_start_end_nodes(edges, nodes)

In [36]:

# find all child nodes with parents that are not dead ends
child_nodes = nodes[(nodes.refmain.notna()) & (nodes.deadend == 0)]

In [37]:
edges["modified"] = False

In [38]:
# assign edges from child nodes with parents to parent nodes
for ix, row in child_nodes.iterrows():
    idx = ix
    # ID of this child node
    this_node_id = row.node_id

    # geometry of the child nodes parent node
    parent_geom = nodes.loc[
        nodes.node_id == int(child_nodes.loc[ix, "refmain"])
    ].geometry.values[0]
    print(f"idx {idx}, step 1")
    
    if parent_geom.distance(row.geometry) > 100:
        continue
    else:
        # all edges which have this child node as their start node
        edges_start = edges.loc[edges.u == this_node_id]

        # all edges which have this child node as their end node
        edges_end = edges.loc[edges.v == this_node_id]
        print(f"idx {idx}, step 2") 
        
        for ix, row in edges_start.iterrows():
            # get coordinate in edge linestring
            edge_coords = list(row.geometry.coords)

            # replace start coordinate (child node) with geometry of parent node
            edge_coords[0] = parent_geom.coords[0]
            
            # create new linestring from updated coordinates
            new_linestring = LineString(edge_coords)

            # update edge geometry
            edges.loc[ix, "geometry"] = new_linestring

            # mark edge as modified
            edges.loc[ix, "modified"] = True
            print(f"idx {idx}, step 3") 
            
        for ix, row in edges_end.iterrows():
            # get coordinate in edge linestring
            edge_coords = list(row.geometry.coords)

            # replace end coordinate (child node) with geometry of parent node
            edge_coords[-1] = parent_geom.coords[0]

            # create new linestring from updated coordinates
            new_linestring = LineString(edge_coords)

            # update edge geometry
            edges.loc[ix, "geometry"] = new_linestring

            # mark edge as modified
            edges.loc[ix, "modified"] = True
            print(f"idx {idx}, step 4")

print("for loop is done")

idx 11, step 1
idx 11, step 2
idx 11, step 3
idx 11, step 3
idx 11, step 4
idx 23, step 1
idx 23, step 2
idx 23, step 3
idx 23, step 3
idx 23, step 4
idx 26, step 1
idx 26, step 2
idx 26, step 3
idx 26, step 4
idx 26, step 4
idx 43, step 1
idx 43, step 2
idx 43, step 3
idx 43, step 4
idx 43, step 4
idx 55, step 1
idx 55, step 2
idx 55, step 3
idx 55, step 4
idx 55, step 4
idx 56, step 1
idx 56, step 2
idx 56, step 3
idx 56, step 4
idx 56, step 4
idx 58, step 1
idx 58, step 2
idx 58, step 3
idx 58, step 3
idx 58, step 4
idx 76, step 1
idx 76, step 2
idx 76, step 3
idx 76, step 3
idx 76, step 4
idx 76, step 4
idx 90, step 1
idx 90, step 2
idx 90, step 3
idx 90, step 3
idx 90, step 3
idx 90, step 4
idx 95, step 1
idx 95, step 2
idx 95, step 3
idx 95, step 4
idx 95, step 4
idx 129, step 1
idx 129, step 2
idx 129, step 3
idx 129, step 4
idx 129, step 4
idx 137, step 1
idx 137, step 2
idx 137, step 3
idx 137, step 3
idx 137, step 4
idx 140, step 1
idx 140, step 2
idx 140, step 3
idx 140, ste

In [39]:
# drop old u,v columns
edges.drop(["u", "v"], axis=1, inplace=True)

# find new start and end nodes
edges = graphedit.assign_edges_start_end_nodes(edges, nodes)

# find edges with same start and end node (but could still be on different roads!)
edges["key"] = 0

# Set u to be to be the smaller one of u,v nodes (based on node id) - to identify parallel edges between u,v / v,u matches
graphedit.order_edge_nodes(edges)

edges = graphedit.find_parallel_edges(edges)

# delete modified edges whith same u and v
edges.drop(
    edges.loc[(edges.u == edges.v) & (edges.modified == True)].index, inplace=True
)

# find parallel edges of approximate same length
edges["drop"] = False
edges["length"] = edges.geometry.length

grouped = edges.groupby(["u", "v"])

for name, group in grouped:
    if len(group) > 1:
        # compare length of all members in groups pair wise
        group_lengths = group.length.to_list()

        duplicates = []
        # mark as duplicate if length difference is less than 20 %
        for i in range(len(group_lengths)):
            for j in range(i + 1, len(group_lengths)):
                if (
                    abs(
                        (
                            (group_lengths[i] - group_lengths[j])
                            / ((group_lengths[i] + group_lengths[j]) / 2)
                        )
                        * 100
                    )
                    < 8
                ):
                    duplicates.append((group_lengths[i], group_lengths[j]))

        length_of_edges_to_drop = set([min(d) for d in duplicates])

        edges.loc[
            group.loc[group.length.isin(length_of_edges_to_drop)].index, "drop"
        ] = True

# nodes used by new network edges
nodes_in_use = nodes.loc[nodes.node_id.isin(set(edges.u.to_list() + edges.v.to_list()))]

# edges without parallel edges
edges_no_parallel = edges.loc[edges["drop"] == False]

In [40]:
assert len(nodes.loc[nodes.ismain == True]) == len(
    nodes_in_use.loc[nodes_in_use.ismain == True]
)

In [41]:
assert len(edges) == len(edges.id.unique())

AssertionError: 

In [42]:
len(edges)

15371

In [43]:
len(edges.id.unique())

424

In [44]:
edges.head()

,id,rating,comment,length,oneway,region,uuid,status,privatenot,surfacenot,geometry,edge_id,modified,u,v,key,drop
0,10556,1,NaN,4157.564041,0,Faaborg-Midtfyn,l10556,Concept,0,0,"LINESTRING (599587.050 6127591.470, 599584.060...",10556,False,5324,9289,0,False
1,12522,2,added due to workshop feedback,1837.683972,0,Faaborg-Midtfyn,l12522,Concept,0,1,"LINESTRING (593544.670 6129398.110, 593555.870...",12522,False,8150,9254,0,False
2,12527,1,added due to workshop feedback,228.928843,1,Faaborg-Midtfyn,l12527,Concept,0,0,"LINESTRING (592219.340 6129322.330, 592232.440...",12527,True,8148,8150,0,True
3,12523,1,Redrawn due to workshop feedback,291.788417,1,Faaborg-Midtfyn,l12523,Concept,0,0,"LINESTRING (592120.600 6128446.640, 592119.260...",12523,False,8145,8146,0,True
4,14596,1,NaN,1216.600359,0,Faaborg-Midtfyn,l14596,Concept,0,0,"LINESTRING (591073.630 6127526.510, 591076.180...",14596,False,8145,9250,0,False


In [ ]:


### SAVE COMMUNICATION NODE AND EDGE DATA TO FILE
# these are the "communication data" layers that will be used by all consecutive scripts FOR PLOTTING


assert len(edges_no_parallel) == len(edges_no_parallel.id.unique())
assert len(nodes_in_use) == len(nodes_in_use.id.unique())
